https://fakeddit.netlify.app

In [2]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time
import argparse


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/guanruijia/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/guanruijia/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/Users/guanruijia/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 725, in start
    self.io

AttributeError: _ARRAY_API not found

In [3]:
df = pd.read_csv('fakeddit/multimodal_train.tsv', sep='\t')

print(f"Dataset: {df.shape}")
print(f"\nrow: {df.columns}")
print(f"\nfirst 5 rows:")
df.head()

Dataset: (564000, 16)

row: Index(['author', 'clean_title', 'created_utc', 'domain', 'hasImage', 'id',
       'image_url', 'linked_submission_id', 'num_comments', 'score',
       'subreddit', 'title', 'upvote_ratio', '2_way_label', '3_way_label',
       '6_way_label'],
      dtype='object')

first 5 rows:


,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,Alexithymia,my walgreens offbrand mucinex was engraved wit...,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdB...,NaN,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved wit...,0.84,1,0,0
1,VIDCAs17,this concerned sink with a tiny hat,1.534727e+09,i.redd.it,True,98pbid,https://preview.redd.it/wsfx0gp0f5h11.jpg?widt...,NaN,2.0,119,pareidolia,This concerned sink with a tiny hat,0.99,0,2,2
2,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFA...,NaN,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
3,NaN,puppy taking in the view,1.471341e+09,i.imgur.com,True,4xypkv,https://external-preview.redd.it/HLtVNhTR6wtYt...,NaN,26.0,250,photoshopbattles,PsBattle: Puppy taking in the view,0.95,1,0,0
4,3rikR3ith,i found a face in my sheet music too,1.525318e+09,i.redd.it,True,8gnet9,https://preview.redd.it/ri7ut2wn8kv01.jpg?widt...,NaN,2.0,13,pareidolia,I found a face in my sheet music too!,0.84,0,2,2


In [4]:
print(df['image_url'].head(10))

0    https://external-preview.redd.it/WylDbZrnbvZdB...
1    https://preview.redd.it/wsfx0gp0f5h11.jpg?widt...
2    https://external-preview.redd.it/6fNhdbc6K1vFA...
3    https://external-preview.redd.it/HLtVNhTR6wtYt...
4    https://preview.redd.it/ri7ut2wn8kv01.jpg?widt...
5    https://external-preview.redd.it/FQ-J9OIPFRpqi...
6    https://preview.redd.it/l9gvkkf3jizy.jpg?width...
7    https://external-preview.redd.it/KHisCPOGwz7cz...
8                     http://i.imgur.com/vbveIEd%2ejpg
9    https://preview.redd.it/31aw9wjucah11.jpg?widt...
Name: image_url, dtype: object


In [ ]:
def download_image(url, save_path, timeout=10):
    """
    Download a single image from a URL and save it to the specified path.
    
    Args:
        url: 
        save_path: 
        timeout: 超时时间（秒）
    
    Returns:
        bool: 下载是否成功
    """
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        img.save(save_path)
        return True
    except Exception as e:
        print(f"下载失败 {url}: {str(e)}")
        return False

def download_images_batch(df, save_dir='fakeddit/images', max_images=None, num_workers=5):
    """
    Download images in batch from URLs in the DataFrame.
    
    Args:
        df: 包含 image_url 列的 DataFrame
        save_dir: 保存目录
        max_images: 最多下载多少张（None 表示全部下载）
        num_workers: 并行下载的线程数
    """
    # 创建保存目录
    os.makedirs(save_dir, exist_ok=True)
    
    # 筛选有效的 URL
    valid_df = df[df['image_url'].notna()].copy()
    if max_images:
        valid_df = valid_df.head(max_images)
    
    print(f"准备下载 {len(valid_df)} 张图片...")
    
    # 准备下载任务
    download_tasks = []
    for idx, row in valid_df.iterrows():
        url = row['image_url']
        # 使用 ID 或索引作为文件名
        file_name = f"{idx}.jpg"
        save_path = os.path.join(save_dir, file_name)
        download_tasks.append((url, save_path))
    
    # 并行下载
    successful = 0
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(
            executor.map(lambda x: download_image(x[0], x[1]), download_tasks),
            total=len(download_tasks),
            desc="下载进度"
        ))
        successful = sum(results)
    
    print(f"\n下载完成！成功: {successful}/{len(download_tasks)}")
    return successful

In [7]:
download_image('https://www.reddit.com/media?url=https%3A%2F%2Fexternal-preview.redd.it%2FWylDbZrnbvZdBpgfa3ntxYf17CBHndiJWHylVm2j_nY.jpg%3Fwidth%3D320%26crop%3Dsmart%26auto%3Dwebp%26s%3D449659a10792de4d55c2f27d2176fdc8bc66e72a', 'test_image.jpg')

下载失败 https://www.reddit.com/media?url=https%3A%2F%2Fexternal-preview.redd.it%2FWylDbZrnbvZdBpgfa3ntxYf17CBHndiJWHylVm2j_nY.jpg%3Fwidth%3D320%26crop%3Dsmart%26auto%3Dwebp%26s%3D449659a10792de4d55c2f27d2176fdc8bc66e72a: cannot identify image file <_io.BytesIO object at 0x1064933d0>


False

In [ ]:
# 调试：查看响应内容
import urllib.parse

test_url = 'https://www.reddit.com/media?url=https%3A%2F%2Fexternal-preview.redd.it%2FWylDbZrnbvZdBpgfa3ntxYf17CBHndiJWHylVm2j_nY.jpg%3Fwidth%3D320%26crop%3Dsmart%26auto%3Dwebp%26s%3D449659a10792de4d55c2f27d2176fdc8bc66e72a'

# 解析 URL，提取真实的图片地址
if 'reddit.com/media?url=' in test_url:
    # 从 URL 参数中提取真实图片地址
    parsed = urllib.parse.urlparse(test_url)
    params = urllib.parse.parse_qs(parsed.query)
    if 'url' in params:
        actual_url = params['url'][0]
        print(f"提取的真实图片 URL: {actual_url}")
        
        # 尝试下载真实 URL
        try:
            response = requests.get(actual_url, timeout=10)
            print(f"状态码: {response.status_code}")
            print(f"Content-Type: {response.headers.get('content-type')}")
            
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                print(f"✓ 成功！图片格式: {img.format}, 大小: {img.size}")
        except Exception as e:
            print(f"✗ 错误: {e}")
else:
    print("不是 Reddit 重定向 URL")

In [ ]:
# 改进的下载函数 - 处理 Reddit 重定向
import urllib.parse

def extract_actual_url(url):
    """
    从 Reddit 重定向 URL 中提取真实的图片地址
    """
    if 'reddit.com/media?url=' in url:
        parsed = urllib.parse.urlparse(url)
        params = urllib.parse.parse_qs(parsed.query)
        if 'url' in params:
            return params['url'][0]
    return url

def download_image_improved(url, save_path, timeout=10):
    """
    改进的图片下载函数，支持 Reddit 重定向
    
    Args:
        url: 图片 URL（可能是 Reddit 重定向）
        save_path: 保存路径
        timeout: 超时时间（秒）
    
    Returns:
        bool: 下载是否成功
    """
    try:
        # 提取真实 URL
        actual_url = extract_actual_url(url)
        
        # 添加 User-Agent 避免被 Reddit 屏蔽
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(actual_url, headers=headers, timeout=timeout)
        response.raise_for_status()
        
        # 检查是否是图片
        content_type = response.headers.get('content-type', '')
        if not content_type.startswith('image/'):
            print(f"警告: {actual_url} 返回的不是图片 (Content-Type: {content_type})")
            return False
        
        # 打开并保存图片
        img = Image.open(BytesIO(response.content))
        img.save(save_path)
        return True
        
    except Exception as e:
        print(f"下载失败 {url}: {str(e)}")
        return False

def download_images_batch_improved(df, save_dir='fakeddit/images', max_images=None, num_workers=5):
    """
    改进的批量下载函数
    
    Args:
        df: 包含 image_url 列的 DataFrame
        save_dir: 保存目录
        max_images: 最多下载多少张（None 表示全部下载）
        num_workers: 并行下载的线程数
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # 筛选有效的 URL
    valid_df = df[df['image_url'].notna()].copy()
    if max_images:
        valid_df = valid_df.head(max_images)
    
    print(f"准备下载 {len(valid_df)} 张图片...")
    
    # 准备下载任务
    download_tasks = []
    for idx, row in valid_df.iterrows():
        url = row['image_url']
        # 使用 id 列作为文件名（如果存在）
        if 'id' in row and pd.notna(row['id']):
            file_name = f"{row['id']}.jpg"
        else:
            file_name = f"{idx}.jpg"
        save_path = os.path.join(save_dir, file_name)
        download_tasks.append((url, save_path))
    
    # 并行下载
    successful = 0
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(
            executor.map(lambda x: download_image_improved(x[0], x[1]), download_tasks),
            total=len(download_tasks),
            desc="下载进度"
        ))
        successful = sum(results)
    
    print(f"\n下载完成！成功: {successful}/{len(download_tasks)}")
    return successful

In [ ]:
# 测试改进的下载函数
test_url = df.iloc[0]['image_url']
print(f"测试 URL: {test_url}")
print(f"真实 URL: {extract_actual_url(test_url)}")

# 测试下载
result = download_image_improved(test_url, 'fakeddit/test_improved.jpg')
if result:
    print("✓ 下载成功！")
    # 显示图片
    img = Image.open('fakeddit/test_improved.jpg')
    print(f"图片大小: {img.size}")
    display(img)